In [38]:
import numpy as np
import pandas as pd
import hvplot.pandas
import datetime as dt
import requests
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from pathlib import Path

In [47]:
races = pd.read_csv('Resources/races1.csv')
results = pd.read_csv('Resources/results1.csv')
qualifying = pd.read_csv('Resources/qualifying.csv')
constructor_standings = pd.read_csv('Resources/constructor_standings1.csv')
driver_standings = pd.read_csv('Resources/driver_standings1.csv')
results

,Unnamed: 0,season,round,circuit_id,driver,date_of_birth,nationality,constructor,grid,time,status,points,podium
0,0,2000,1,albert_park,michael_schumacher,1969-01-03,German,ferrari,3,5641987.0,Finished,10.0,1
1,1,2000,1,albert_park,barrichello,1972-05-23,Brazilian,ferrari,4,5653402.0,Finished,6.0,2
2,2,2000,1,albert_park,ralf_schumacher,1975-06-30,German,williams,11,5661996.0,Finished,4.0,3
3,3,2000,1,albert_park,villeneuve,1971-04-09,Canadian,bar,8,5686434.0,Finished,3.0,4
4,4,2000,1,albert_park,fisichella,1973-01-14,Italian,benetton,9,5687152.0,Finished,2.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8715,8715,2021,22,yas_marina,latifi,1995-06-29,Canadian,williams,16,NaN,Accident,0.0,16
8716,8716,2021,22,yas_marina,giovinazzi,1993-12-14,Italian,alfa,14,NaN,Gearbox,0.0,17
8717,8717,2021,22,yas_marina,russell,1998-02-15,British,williams,17,NaN,Gearbox,0.0,18
8718,8718,2021,22,yas_marina,raikkonen,1979-10-17,Finnish,alfa,18,NaN,Brakes,0.0,19


In [48]:
races

,Unnamed: 0,season,round,circuit_id,lat,long,country,date,url
0,0,2000,1,albert_park,-37.8497,144.96800,Australia,2000-03-12,http://en.wikipedia.org/wiki/2000_Australian_G...
1,1,2000,2,interlagos,-23.7036,-46.69970,Brazil,2000-03-26,http://en.wikipedia.org/wiki/2000_Brazilian_Gr...
2,2,2000,3,imola,44.3439,11.71670,Italy,2000-04-09,http://en.wikipedia.org/wiki/2000_San_Marino_G...
3,3,2000,4,silverstone,52.0786,-1.01694,UK,2000-04-23,http://en.wikipedia.org/wiki/2000_British_Gran...
4,4,2000,5,catalunya,41.5700,2.26111,Spain,2000-05-07,http://en.wikipedia.org/wiki/2000_Spanish_Gran...
...,...,...,...,...,...,...,...,...,...
406,406,2021,18,rodriguez,19.4042,-99.09070,Mexico,2021-11-07,http://en.wikipedia.org/wiki/2021_Mexican_Gran...
407,407,2021,19,interlagos,-23.7036,-46.69970,Brazil,2021-11-14,http://en.wikipedia.org/wiki/2021_S%C3%A3o_Pau...
408,408,2021,20,losail,25.4900,51.45420,Qatar,2021-11-21,http://en.wikipedia.org/wiki/2021_Qatar_Grand_...
409,409,2021,21,jeddah,21.6319,39.10440,Saudi Arabia,2021-12-05,http://en.wikipedia.org/wiki/2021_Saudi_Arabia...


In [49]:
qualifying

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236
...,...,...,...,...,...,...,...,...,...
9190,9248,1076,848,3,23,16,1:20.135,NaN,NaN
9191,9249,1076,825,210,20,17,1:20.254,NaN,NaN
9192,9250,1076,20,117,5,18,1:21.149,NaN,NaN
9193,9251,1076,849,3,6,19,1:21.372,NaN,NaN


In [50]:
df1 = pd.DataFrame(races)


df1

,Unnamed: 0,season,round,circuit_id,lat,long,country,date,url
0,0,2000,1,albert_park,-37.8497,144.96800,Australia,2000-03-12,http://en.wikipedia.org/wiki/2000_Australian_G...
1,1,2000,2,interlagos,-23.7036,-46.69970,Brazil,2000-03-26,http://en.wikipedia.org/wiki/2000_Brazilian_Gr...
2,2,2000,3,imola,44.3439,11.71670,Italy,2000-04-09,http://en.wikipedia.org/wiki/2000_San_Marino_G...
3,3,2000,4,silverstone,52.0786,-1.01694,UK,2000-04-23,http://en.wikipedia.org/wiki/2000_British_Gran...
4,4,2000,5,catalunya,41.5700,2.26111,Spain,2000-05-07,http://en.wikipedia.org/wiki/2000_Spanish_Gran...
...,...,...,...,...,...,...,...,...,...
406,406,2021,18,rodriguez,19.4042,-99.09070,Mexico,2021-11-07,http://en.wikipedia.org/wiki/2021_Mexican_Gran...
407,407,2021,19,interlagos,-23.7036,-46.69970,Brazil,2021-11-14,http://en.wikipedia.org/wiki/2021_S%C3%A3o_Pau...
408,408,2021,20,losail,25.4900,51.45420,Qatar,2021-11-21,http://en.wikipedia.org/wiki/2021_Qatar_Grand_...
409,409,2021,21,jeddah,21.6319,39.10440,Saudi Arabia,2021-12-05,http://en.wikipedia.org/wiki/2021_Saudi_Arabia...


In [51]:
df2 = pd.merge(df1, results, how='inner', 
               on=['season', 'round', 'circuit_id']).drop(['url','points', 'status', 'time'], 
                                                                 axis = 1)
df2

,Unnamed: 0_x,season,round,circuit_id,lat,long,country,date,Unnamed: 0_y,driver,date_of_birth,nationality,constructor,grid,podium
0,0,2000,1,albert_park,-37.8497,144.9680,Australia,2000-03-12,0,michael_schumacher,1969-01-03,German,ferrari,3,1
1,0,2000,1,albert_park,-37.8497,144.9680,Australia,2000-03-12,1,barrichello,1972-05-23,Brazilian,ferrari,4,2
2,0,2000,1,albert_park,-37.8497,144.9680,Australia,2000-03-12,2,ralf_schumacher,1975-06-30,German,williams,11,3
3,0,2000,1,albert_park,-37.8497,144.9680,Australia,2000-03-12,3,villeneuve,1971-04-09,Canadian,bar,8,4
4,0,2000,1,albert_park,-37.8497,144.9680,Australia,2000-03-12,4,fisichella,1973-01-14,Italian,benetton,9,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8715,410,2021,22,yas_marina,24.4672,54.6031,UAE,2021-12-12,8715,latifi,1995-06-29,Canadian,williams,16,16
8716,410,2021,22,yas_marina,24.4672,54.6031,UAE,2021-12-12,8716,giovinazzi,1993-12-14,Italian,alfa,14,17
8717,410,2021,22,yas_marina,24.4672,54.6031,UAE,2021-12-12,8717,russell,1998-02-15,British,williams,17,18
8718,410,2021,22,yas_marina,24.4672,54.6031,UAE,2021-12-12,8718,raikkonen,1979-10-17,Finnish,alfa,18,19


In [52]:
df3 = pd.merge(df2, driver_standings, how='left', 
               on=['season', 'round', 'driver']) 
df3

,Unnamed: 0_x,season,round,circuit_id,lat,long,country,date,Unnamed: 0_y,driver,date_of_birth,nationality,constructor,grid,podium,Unnamed: 0,driver_points,driver_wins,driver_standings_pos
0,0,2000,1,albert_park,-37.8497,144.9680,Australia,2000-03-12,0,michael_schumacher,1969-01-03,German,ferrari,3,1,0.0,0.0,0.0,0.0
1,0,2000,1,albert_park,-37.8497,144.9680,Australia,2000-03-12,1,barrichello,1972-05-23,Brazilian,ferrari,4,2,1.0,0.0,0.0,0.0
2,0,2000,1,albert_park,-37.8497,144.9680,Australia,2000-03-12,2,ralf_schumacher,1975-06-30,German,williams,11,3,2.0,0.0,0.0,0.0
3,0,2000,1,albert_park,-37.8497,144.9680,Australia,2000-03-12,3,villeneuve,1971-04-09,Canadian,bar,8,4,3.0,0.0,0.0,0.0
4,0,2000,1,albert_park,-37.8497,144.9680,Australia,2000-03-12,4,fisichella,1973-01-14,Italian,benetton,9,5,4.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8715,410,2021,22,yas_marina,24.4672,54.6031,UAE,2021-12-12,8715,latifi,1995-06-29,Canadian,williams,16,16,9099.0,7.0,0.0,17.0
8716,410,2021,22,yas_marina,24.4672,54.6031,UAE,2021-12-12,8716,giovinazzi,1993-12-14,Italian,alfa,14,17,9100.0,3.0,0.0,18.0
8717,410,2021,22,yas_marina,24.4672,54.6031,UAE,2021-12-12,8717,russell,1998-02-15,British,williams,17,18,9097.0,16.0,0.0,15.0
8718,410,2021,22,yas_marina,24.4672,54.6031,UAE,2021-12-12,8718,raikkonen,1979-10-17,Finnish,alfa,18,19,9098.0,10.0,0.0,16.0


In [53]:
df4 = pd.merge(df3, constructor_standings, how='left', 
               on=['season', 'round', 'constructor']) 

df4

C:\Users\triba\Anaconda3\envs\algotrading\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Unnamed: 0_x'} in the result is deprecated and will raise a MergeError in a future version.
  


,Unnamed: 0_x,season,round,circuit_id,lat,long,country,date,Unnamed: 0_y,driver,...,grid,podium,Unnamed: 0_x,driver_points,driver_wins,driver_standings_pos,Unnamed: 0_y,constructor_points,constructor_wins,constructor_standings_pos
0,0,2000,1,albert_park,-37.8497,144.9680,Australia,2000-03-12,0,michael_schumacher,...,3,1,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,0,2000,1,albert_park,-37.8497,144.9680,Australia,2000-03-12,1,barrichello,...,4,2,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,0,2000,1,albert_park,-37.8497,144.9680,Australia,2000-03-12,2,ralf_schumacher,...,11,3,2.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,0,2000,1,albert_park,-37.8497,144.9680,Australia,2000-03-12,3,villeneuve,...,8,4,3.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,0,2000,1,albert_park,-37.8497,144.9680,Australia,2000-03-12,4,fisichella,...,9,5,4.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8715,410,2021,22,yas_marina,24.4672,54.6031,UAE,2021-12-12,8715,latifi,...,16,16,9099.0,7.0,0.0,17.0,2900.0,23.0,0.0,8.0
8716,410,2021,22,yas_marina,24.4672,54.6031,UAE,2021-12-12,8716,giovinazzi,...,14,17,9100.0,3.0,0.0,18.0,2901.0,13.0,0.0,9.0
8717,410,2021,22,yas_marina,24.4672,54.6031,UAE,2021-12-12,8717,russell,...,17,18,9097.0,16.0,0.0,15.0,2900.0,23.0,0.0,8.0
8718,410,2021,22,yas_marina,24.4672,54.6031,UAE,2021-12-12,8718,raikkonen,...,18,19,9098.0,10.0,0.0,16.0,2901.0,13.0,0.0,9.0


In [54]:
#we need to get qualifying1 from the other notebook(last step loading forever)

final_df = pd.merge(df4, qualifying, how='inner', 
                    on=['season', 'round', 'grid']).drop(['driver_name', 'car'], 
                                                         axis = 1)

KeyError: 'season'